In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from captum.attr import IntegratedGradients

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from sklearn.model_selection import train_test_split

# Captum for BertForSequenceClassification (multi label)

## Prepare dataset

In [4]:
post_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_post.csv', encoding='UTF-8')
comment_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_comment.csv', encoding='UTF-8')

In [5]:
# texts (x)
post_contents = list(post_df['content'])
comment_bodies = list(comment_df['content'])

# satisfaction score (y)
satisfactions_float = list(post_df['satisfaction'])
satisfactions = []

for s in satisfactions_float:
    if s < 3.5:
        satisfactions.append(0)
    elif s < 5:
        satisfactions.append(1)
    else:
        satisfactions.append(2)

data = []

for content, body, satisfaction in zip(post_contents, comment_bodies, satisfactions):
    data.append([content + '[SEP]' + body, satisfaction])

columns = ['contents', 'label']
df = pd.DataFrame(data, columns=columns)

# data split (train & test sets)
idx_train, idx_remain = train_test_split(df.index.values, test_size=0.20, random_state=42)
idx_val, idx_test = train_test_split(idx_remain, test_size=0.50, random_state=42)

print(idx_train.shape, idx_val.shape, idx_test.shape)

train_df = df.iloc[idx_train]
val_df = df.iloc[idx_val]
test_df = df.iloc[idx_test]

count_min_label = min(train_df['label'].value_counts())

labels = [0, 1, 2]

train_sample_df = pd.DataFrame([], columns=columns)

for label in labels:
    tmp = train_df[train_df['label'] == label]
    tmp_sampled = tmp.sample(frac=1).iloc[:count_min_label]
    train_sample_df = pd.concat([train_sample_df, tmp_sampled])

(800,) (100,) (100,)


## Load model

In [6]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [84]:
model_path = f'../predicting-satisfaction-using-graphs/model/epoch_10.model'
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3, problem_type='multi_label_classification', output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [85]:
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [9]:
model.eval()
model.zero_grad()

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

## Define custom forward function

In [11]:
def predict(inputs, token_type_ids=None, attention_mask=None):
    inputs = {'input_ids': inputs,
             'token_type_ids': token_type_ids,
             'attention_mask': attention_mask}
    output = model(**inputs)
        
    return output.logits

In [12]:
def classification_forward_func(inputs, token_type_ids=None, attention_mask=None):
    print(attention_mask)
    pred = predict(inputs, token_type_ids, attention_mask)
    # pred = np.argmax(pred.cpu().detach().numpy(), axis=1)
    # return torch.tensor(pred, device=device)
    return pred
    # return torch.softmax(pred, dim = 1)[0][1].unsqueeze(0)

## Define inputs & baselines

In [13]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference (zero padding)
sep_token_id = tokenizer.sep_token_id # A token used as a separator between post and comment and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated post-comment word sequence

In [14]:
print(ref_token_id, sep_token_id, cls_token_id)

0 102 101


In [15]:
def construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id):
    post_ids = tokenizer.encode(post, add_special_tokens=False)
    comment_ids = tokenizer.encode(comment, add_special_tokens=False)

    # construct input token ids
    input_ids = torch.tensor([[cls_token_id] + post_ids + [sep_token_id] + comment_ids + [sep_token_id]], device=device)
    seq_len = input_ids.size(1)
    
    # token_type_ids is used both embeddings
    token_type_ids = torch.tensor([[0 for i in range(seq_len)]], device=device)

    # construct reference token ids 
    ref_input_ids = torch.tensor([[cls_token_id] + [ref_token_id] * len(post_ids) + [sep_token_id] + \
        [ref_token_id] * len(comment_ids) + [sep_token_id]], device=device)

    return input_ids, ref_input_ids, token_type_ids

In [16]:
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [17]:
def construct_whole_bert_embeddings(input_ids, ref_input_ids, token_type_ids):
    input_embeddings = model.bert.embeddings(input_ids, token_type_ids=token_type_ids)
    ref_input_embeddings = model.bert.embeddings(ref_input_ids, token_type_ids=token_type_ids)
    
    return input_embeddings, ref_input_embeddings

In [18]:
post, comment, true_label = post_contents[0], comment_bodies[0], satisfactions[0]

In [19]:
print(construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id))

(tensor([[  101,  2353,  2095,  2267,  3076,  1012,  2026,  6245,  2038,  2467,
         27674,  4321,  2013,  2591, 10089,  1010, 10261,  1045,  2018,  2814,
          2021,  2025,  4209,  2129,  2000,  2191,  2068,  1012,  1045,  2985,
          2048,  2086,  1999, 19568,  2015,  1998,  2196,  3764,  2000,  2151,
          1997,  2026, 10638,  1998,  2134,  1005,  1056,  2428,  3113, 10334,
          1012,  2197,  2305,  1010,  2005,  1996,  2034,  2051,  1999,  2026,
          2166,  1010,  1045,  2001,  4778,  2000,  1037,  2283,  1006,  2025,
          7714,  2568,  2017,  1025,  1045,  2001,  1037,  2112,  1997,  1037,
          2177,  2008,  2288, 13643,  4778,  1007,  1012,  1045, 10749,  1037,
          2210,  2978,  1010,  2021,  2288, 10247,  2100,  2855,  1010,  1998,
          2025,  4209,  2026,  6537,  1045,  3030,  1012,  1045,  2001,  3294,
         17358,  2306,  1996,  2034,  3178,  1012,  4661, 25795,  1998,  2019,
          8138, 14806,  1010,  1996,  2069,  2111, 

In [20]:
input_ids, ref_input_ids, token_type_ids = construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id)

In [21]:
attention_mask = construct_attention_mask(input_ids)

In [22]:
indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [23]:
input_embeddings, ref_input_embeddings = construct_whole_bert_embeddings(input_ids, ref_input_ids, token_type_ids)

In [24]:
print(input_embeddings.shape)

torch.Size([1, 285, 768])


In [25]:
len(all_tokens)

285

In [26]:
len(input_ids[0])

285

## Predicting

In [27]:
pred = predict(input_ids,
               token_type_ids=token_type_ids,
               attention_mask=attention_mask)

print(pred)

tensor([[ 4.4980, -4.1010, -5.0655]], grad_fn=<AddmmBackward>)


In [28]:
pred = classification_forward_func(input_ids,
               token_type_ids=token_type_ids,
               attention_mask=attention_mask)

print(pred)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[ 4.4980, -4.1010, -5.0655]

## Apply LayerIntegratedGradients

In [29]:
lig = LayerIntegratedGradients(classification_forward_func, model.bert.embeddings)

In [30]:
print(ref_input_ids)

tensor([[101,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0

In [31]:
print(ref_input_embeddings)

tensor([[[ 0.1680, -0.2795, -0.3270,  ..., -0.0290,  0.0403,  0.1622],
         [ 0.1687,  0.2632, -0.2931,  ...,  0.3789,  0.5761, -0.2864],
         [-0.2422,  0.1995, -0.1237,  ...,  0.1360,  0.3750, -0.3164],
         ...,
         [ 0.3701,  0.1432,  0.4939,  ..., -0.2050, -0.1995, -0.5937],
         [ 0.3037,  0.5574,  0.2793,  ..., -0.2611, -0.3623, -0.5703],
         [-0.0421,  0.2734,  0.2053,  ..., -0.6526,  0.0529, -0.4776]]],
       grad_fn=<NativeLayerNormBackward>)


In [32]:
attribution, attribution_delta = lig.attribute(inputs=input_ids, 
                                               baselines=ref_input_ids,
                                               target=2,
                                               additional_forward_args=(token_type_ids, attention_mask),
                                               return_convergence_delta=True)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1,

In [31]:
print(input_embeddings)
print(attribution)

tensor([[[ 0.1680, -0.2795, -0.3270,  ..., -0.0290,  0.0403,  0.1622],
         [ 0.4657, -0.3681,  0.3283,  ..., -0.4469, -0.6128, -0.8702],
         [ 0.7787,  0.6141, -0.7667,  ...,  0.2643,  0.6185, -0.6873],
         ...,
         [-0.3731, -0.1633,  0.4561,  ...,  0.2998,  0.4657,  0.0302],
         [-0.0099,  0.5589,  0.1702,  ...,  0.1803,  0.3020,  0.3123],
         [-0.0421,  0.2734,  0.2053,  ..., -0.6526,  0.0529, -0.4776]]],
       grad_fn=<NativeLayerNormBackward>)
tensor([[[ 0.0000e+00, -0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          -0.0000e+00, -0.0000e+00],
         [ 1.4611e-04,  1.3571e-03,  9.8245e-04,  ...,  5.2328e-05,
           1.9038e-03,  1.2555e-03],
         [-4.8884e-04, -6.8524e-04,  5.1778e-04,  ...,  2.0780e-04,
           1.9909e-04, -1.5710e-05],
         ...,
         [-1.3424e-03,  2.6367e-04, -1.6980e-05,  ...,  3.6467e-05,
          -1.5894e-03,  1.4697e-03],
         [-1.3190e-03, -6.4167e-06,  2.8747e-04,  ..., -2.0616e-04,
           2.0

In [32]:
input_embeddings = input_embeddings.squeeze(0)
print(input_embeddings.shape)

torch.Size([285, 768])


In [33]:
attribution.shape

torch.Size([1, 285, 768])

In [34]:
print(torch.sum(input_embeddings))
print(torch.sum(attribution))

tensor(3377.5234, grad_fn=<SumBackward0>)
tensor(-3.9568, dtype=torch.float64)


In [35]:
print(torch.sum(input_embeddings, axis=1)[0].item())
print(torch.sum(attribution[0], axis=1)[0].item())

-23.016847610473633
0.0


In [36]:
torch.sum(attribution, axis=1).shape
torch.sum(input_embeddings, axis=1).shape

torch.Size([285])

In [37]:
attribution_delta

tensor([-0.1639], dtype=torch.float64)

In [38]:
def summarize_attributions(attribution):
    attributions = attribution.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [39]:
attributions = summarize_attributions(attribution)

In [40]:
print(torch.sum(attributions))

tensor(-4.7069, dtype=torch.float64)


In [37]:
print(attributions.shape)

torch.Size([285])


In [46]:
print(torch.sum(input_embeddings))
print(torch.sum(ref_input_embeddings))
print(torch.sum(attributions))

tensor(3377.5234, grad_fn=<SumBackward0>)
tensor(2551.9597, grad_fn=<SumBackward0>)
tensor(-4.7069, dtype=torch.float64)


## Visualization

In [31]:
score_vis = viz.VisualizationDataRecord(
                        attributions,
                        torch.max(torch.softmax(pred, dim=0)),
                        torch.argmax(pred),  # predicted label
                        true_label,  # true label
                        post,
                        attributions.sum(),       
                        all_tokens,
                        attribution_delta)

## Main function of Captum for multi label classification (LayerIntegratedGradients)

In [43]:
def captum_for_multi_label_classification_lig_attr(post, comment, true_label):
    input_ids, ref_input_ids, token_type_ids = construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id)
    attention_mask = construct_attention_mask(input_ids)
    
    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)

    pred = predict(input_ids,
                   token_type_ids=token_type_ids,
                   attention_mask=attention_mask)
    
    print(input_ids)

    
    for target in range(3):
        attribution, attribution_delta = lig.attribute(inputs=input_ids, 
                                                       baselines=ref_input_ids,
                                                       target=target,
                                                       additional_forward_args=(token_type_ids, attention_mask),
                                                       return_convergence_delta=True)
    
        attributions = summarize_attributions(attribution)
        print(torch.sum(attributions))

In [33]:
post1, comment1, true_label1 = post_contents[3], comment_bodies[3], satisfactions[3]
post2, comment2, true_label2 = post_contents[4], comment_bodies[4], satisfactions[4]

In [45]:
captum_for_multi_label_classification_lig_attr(post1, comment1, true_label1)
captum_for_multi_label_classification_lig_attr(post2, comment2, true_label2)

tensor([[  101,  2471,  2296,  2154,  2005,  2706,  1010,  1045,  1005,  2310,
          2908,  2000,  1037,  3595,  6318,  2173,  1999,  1996,  3224,  2379,
          2026,  2160,  1012,  2013,  2028,  1997,  1996,  3628,  1010, 17991,
          1037,  8164,  2007,  1037,  2053,  9232,  1012,  1045,  1005,  2310,
          2641,  2008,  1996, 25551,  2126,  1010,  1998,  1996,  2625,  9145,
          2126,  1012,  1045,  2572, 12422,  2065,  1045,  1005,  1049,  5214,
          1997,  2725,  1996, 15046,  1010,  2021,  2296,  7292,  1045,  1005,
          1049,  2045,  1010,  1045,  2074,  4133,  2104,  1996,  3392,  1998,
         10720,  1012,  2008,  1005,  1055,  2035,  1012,  2138,  1997,  1996,
          2477,  3141,  2000,  2026,  2034,  1013,  1054,  1013,  6245,  2695,
          1010,  1998,  2070,  2060, 19662,  6392,  2477,  1012,   102,  2059,
          2017,  2031,  2498,  2000,  4558,  1012,  2425,  2014, 12043,  1012,
           102]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [62]:
def captum_for_multi_label_classification_lig(post, comment, true_label):
    input_ids, ref_input_ids, token_type_ids = construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id)
    attention_mask = construct_attention_mask(input_ids)
    
    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)

    pred = predict(input_ids,
                   token_type_ids=token_type_ids,
                   attention_mask=attention_mask)
    
    print(input_ids)

    
    for target in range(3):
        attribution, attribution_delta = lig.attribute(inputs=input_ids, 
                                                       baselines=ref_input_ids,
                                                       target=target,
                                                       additional_forward_args=(token_type_ids, attention_mask),
                                                       return_convergence_delta=True)
    
        attributions = summarize_attributions(attribution)
    
    
        score_vis = viz.VisualizationDataRecord(attributions,
                                                torch.max(torch.softmax(pred, dim=0)),
                                                torch.argmax(pred),  # predicted label
                                                true_label,  # true label
                                                post + ' ' + comment,
                                                attributions.sum(),
                                                all_tokens,
                                                attribution_delta)
    
        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])

In [63]:
post, comment, true_label = post_contents[3], comment_bodies[3], satisfactions[3]

In [65]:
captum_for_multi_label_classification_lig(post, comment, true_label)

tensor([[  101,  2471,  2296,  2154,  2005,  2706,  1010,  1045,  1005,  2310,
          2908,  2000,  1037,  3595,  6318,  2173,  1999,  1996,  3224,  2379,
          2026,  2160,  1012,  2013,  2028,  1997,  1996,  3628,  1010, 17991,
          1037,  8164,  2007,  1037,  2053,  9232,  1012,  1045,  1005,  2310,
          2641,  2008,  1996, 25551,  2126,  1010,  1998,  1996,  2625,  9145,
          2126,  1012,  1045,  2572, 12422,  2065,  1045,  1005,  1049,  5214,
          1997,  2725,  1996, 15046,  1010,  2021,  2296,  7292,  1045,  1005,
          1049,  2045,  1010,  1045,  2074,  4133,  2104,  1996,  3392,  1998,
         10720,  1012,  2008,  1005,  1055,  2035,  1012,  2138,  1997,  1996,
          2477,  3141,  2000,  2026,  2034,  1013,  1054,  1013,  6245,  2695,
          1010,  1998,  2070,  2060, 19662,  6392,  2477,  1012,   102,  2059,
          2017,  2031,  2498,  2000,  4558,  1012,  2425,  2014, 12043,  1012,
           102]])
 Visualization For Score 


 Visualization For Score 


 Visualization For Score 


In [36]:
contents_test = list(test_df['contents'])

posts_test, comments_test = [], []
for content in contents_test:
    post_and_comment = content.split('[SEP]')
    posts_test.append(post_and_comment[0])
    comments_test.append(post_and_comment[1])

satisfactions_test = list(test_df['label'])

In [37]:
count = 0

for post, comment, true_label in zip(posts_test, comments_test, satisfactions_test):
    input_ids, ref_input_ids, token_type_ids = construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id)
    attention_mask = construct_attention_mask(input_ids)
    pred = predict(input_ids,
                   token_type_ids=token_type_ids,
                   attention_mask=attention_mask)
    
    if torch.argmax(pred).item() == true_label:
        count += 1
        
print(count)

41


In [38]:
post, comment, true_label = posts_test[3], comments_test[3], satisfactions_test[3]

In [41]:
captum_for_multi_label_classification(post, comment, true_label)

 Visualization For Score 


 Visualization For Score 


 Visualization For Score 


In [449]:
model.bert.encoder

BertEncoder(
  (layer): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=3072, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

## Main function of Captum for multi label classification (IntegratedGradients)

In [180]:
import torch.nn.functional as F

def classification_forward_func_ig(inputs, attention_mask=None):
    head_mask = [None] * model.bert.config.num_hidden_layers
    encoder_outputs = model.bert.encoder(inputs,
                                         attention_mask=attention_mask[0],
                                         head_mask=head_mask)
    
    sequence_output = encoder_outputs[0]
    pooled_output = model.bert.pooler(sequence_output)
    pooled_output = F.relu(pooled_output)
    # pooled_output = model.dropout(pooled_output)
    logits = model.classifier(pooled_output)
    return logits

In [181]:
model.classifier

Linear(in_features=768, out_features=3, bias=True)

In [182]:
ig = IntegratedGradients(classification_forward_func_ig)

In [183]:
def captum_for_multi_label_classification_ig(post, comment, true_label):
    input_ids, ref_input_ids, token_type_ids = construct_input_ref_pair(post, comment, ref_token_id, sep_token_id, cls_token_id)
    attention_mask = construct_attention_mask(input_ids)
    ref_attention_mask = construct_attention_mask(ref_input_ids)
    
    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)

    pred = predict(input_ids,
                   token_type_ids=token_type_ids,
                   attention_mask=attention_mask)
    
    input_embedding = model.bert.embeddings(input_ids,
                                            token_type_ids=token_type_ids)
    ref_input_embedding = model.bert.embeddings(ref_input_ids,
                                               token_type_ids=token_type_ids)
    
    # input_embedding = input_embedding.squeeze(0)
    # ref_input_embedding = ref_input_embedding.squeeze(0)
    
    # print(torch.sum(input_embedding)-torch.sum(ref_input_embedding))
    
    # print(torch.sum(classification_forward_func_ig(input_embedding, attention_mask)))
    # print(torch.sum(classification_forward_func_ig(ref_input_embedding, ref_attention_mask)))
    
    fi = classification_forward_func_ig(input_embedding, attention_mask)
    fb = classification_forward_func_ig(ref_input_embedding, ref_attention_mask)
    
    a, d = ig.attribute(inputs=input_embedding, target=0, additional_forward_args=attention_mask, return_convergence_delta=True)

    print(torch.sum(fi) - torch.sum(fb))
    print(torch.sum(a))
    return
    
    print(type(fi[0]), type(fb), type(a), type(d))
    print(fi[0].squeeze(0).shape, fb.squeeze(0).shape, a.squeeze(0).shape, d.shape)
    return
    
    print(torch.sum(ig.attribute(inputs=input_embedding,
                                                      target=1,
                                                      additional_forward_args=attention_mask,
                                                      return_convergence_delta=True)[0]))
    return
    
    
    
    for target in range(3):
        attribution, attribution_delta = ig.attribute(inputs=input_embedding,
                                                      target=target,
                                                      additional_forward_args=attention_mask,
                                                      return_convergence_delta=True)
    
        attributions = summarize_attributions(attribution)
    
    
        score_vis = viz.VisualizationDataRecord(attributions,
                                                torch.max(torch.softmax(pred, dim=0)),
                                                torch.argmax(pred),  # predicted label
                                                true_label,  # true label
                                                post + ' ' + comment,
                                                attributions.sum(),
                                                all_tokens,
                                                attribution_delta)
    
        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])

In [184]:
captum_for_multi_label_classification_ig(post1, comment1, true_label1)

tensor(-0.0517, grad_fn=<SubBackward0>)
tensor(12.7541, dtype=torch.float64, grad_fn=<SumBackward0>)


In [502]:
captum_for_multi_label_classification_ig(post, comment, true_label)

 Visualization For Score 


 Visualization For Score 


 Visualization For Score 
